In [1]:
%%configure -f
{ "conf":{
"spark.pyspark.python": "python3",
"spark.pyspark.virtualenv.enabled": "true",
"spark.pyspark.virtualenv.type":"native",
"spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
}}

In [2]:
spark.sparkContext.install_pypi_package('boto3')

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
4,application_1732509690730_0005,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
%pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install pandas>=1.0.5

Note: you may need to restart the kernel to use updated packages.


In [47]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, count, to_date
from pyspark.sql.types import IntegerType
from pyspark.sql import SparkSession
import boto3
from io import StringIO

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
spark = SparkSession.builder.appName("CovidAnalisis").getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
s3_file_path = "s3a://sboterodatasets/covid19/Casos_positivos_de_COVID-19_en_Colombia-1K.csv"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df = spark.read.csv(s3_file_path, header=True, inferSchema=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# 2.1. Columnas
df.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['fecha reporte web', 'ID de caso', 'Fecha de notificación', 'Código DIVIPOLA departamento', 'Nombre departamento', 'Código DIVIPOLA municipio', 'Nombre municipio', 'Edad', 'Unidad de medida de edad', 'Sexo', 'Tipo de contagio', 'Ubicación del caso', 'Estado', 'Código ISO del país', 'Nombre del país', 'Recuperado', 'Fecha de inicio de síntomas', 'Fecha de muerte', 'Fecha de diagnóstico', 'Fecha de recuperación', 'Tipo de recuperación', 'Pertenencia étnica', 'Nombre del grupo étnico']

In [10]:
# 2.2. Tipo de datos
df.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- fecha reporte web: string (nullable = true)
 |-- ID de caso: integer (nullable = true)
 |-- Fecha de notificación: string (nullable = true)
 |-- Código DIVIPOLA departamento: integer (nullable = true)
 |-- Nombre departamento: string (nullable = true)
 |-- Código DIVIPOLA municipio: integer (nullable = true)
 |-- Nombre municipio: string (nullable = true)
 |-- Edad: integer (nullable = true)
 |-- Unidad de medida de edad: integer (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Tipo de contagio: string (nullable = true)
 |-- Ubicación del caso: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- Código ISO del país: integer (nullable = true)
 |-- Nombre del país: string (nullable = true)
 |-- Recuperado: string (nullable = true)
 |-- Fecha de inicio de síntomas: string (nullable = true)
 |-- Fecha de muerte: string (nullable = true)
 |-- Fecha de diagnóstico: string (nullable = true)
 |-- Fecha de recuperación: string (nullable = true)
 |-- Tipo de r

In [11]:
# 2.3. Seleccionar columnas
new_def = df.select("Edad", "Sexo", "Nombre departamento")
new_def

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[Edad: int, Sexo: string, Nombre departamento: string]

In [12]:
# 2.4. Renombrar columnas
new_def = new_def.withColumnRenamed("Nombre departamento", "Departamento")
new_def.columns

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Edad', 'Sexo', 'Departamento']

In [13]:
# 2.5. Agregar columnas
new_df = new_def.withColumn("Año nacimiento", 2020 - col("Edad"))
new_df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[Edad: int, Sexo: string, Departamento: string, A?o nacimiento: int]

In [14]:
# 2.6. Borrar columnas
new_df = new_df.drop("Sexo")
new_df.columns 

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Edad', 'Departamento', 'A?o nacimiento']

In [15]:
# 2.7. Filtrar datos
new_def = new_df.filter(col("Edad") > 50)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# 2.8. Ejecutar alguna función UDF o lambda sobre alguna columna creando una nueva.
def double_age(value):
    return value * 2

# Registrar la UDF
double_udf = udf(double_age, IntegerType())

# Aplicar la UDF a una columna
new_df = new_df.withColumn("Edad doble", double_udf(col("Edad")))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
s3_client = boto3.client('s3')
df_pandas = new_df.toPandas()
csv_buffer = StringIO()
df_pandas.to_csv(csv_buffer)
s3_client.put_object(Bucket='sboterodatasets', Key='covid_jupyter.csv', Body=csv_buffer.getvalue())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': 'Z4WSKPMCVANRXJJZ', 'HostId': 'HvtfxNHWxoigf+s49h/L8tIoH2nl6N6P+Ew2bfPQUsOU/94ITGUBswdPYk/Uj2ygcaKV/QuxBXI=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'HvtfxNHWxoigf+s49h/L8tIoH2nl6N6P+Ew2bfPQUsOU/94ITGUBswdPYk/Uj2ygcaKV/QuxBXI=', 'x-amz-request-id': 'Z4WSKPMCVANRXJJZ', 'date': 'Mon, 25 Nov 2024 05:04:01 GMT', 'x-amz-server-side-encryption': 'AES256', 'etag': '"4f34758bf1c3e1631cf309dbbf7b4bb9"', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'ETag': '"4f34758bf1c3e1631cf309dbbf7b4bb9"', 'ServerSideEncryption': 'AES256'}

In [50]:
# 3.1.  10 departamentos con más casos de covid en Colombia ordenados de mayor a menor.
df.columns
df_grouped = df.groupBy("Nombre departamento").agg(count("ID de caso").alias("Total casos"))

df_sorted = df_grouped.orderBy(col("Total casos").desc())

df_top_10 = df_sorted.limit(10)
df_top_10.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-----------+
|Nombre departamento|Total casos|
+-------------------+-----------+
|             BOGOTA|        401|
|              VALLE|        148|
|          ANTIOQUIA|        106|
|       CUNDINAMARCA|         49|
|          CARTAGENA|         39|
|          RISARALDA|         34|
|       BARRANQUILLA|         31|
|              HUILA|         30|
|            QUINDIO|         23|
|    NORTE SANTANDER|         19|
+-------------------+-----------+

In [51]:
df_pandas = df_top_10.toPandas()
csv_buffer = StringIO()
df_pandas.to_csv(csv_buffer)
s3_client.put_object(Bucket='sboterodatasets', Key='top10_dep_jupyter.csv', Body=csv_buffer.getvalue())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': 'JWM7XZSHVEEM884P', 'HostId': 'HqzydB/ZjvAHG6hGKJXNX4Z1N5hwjhur4PgrntomRGPFl8ygMqpJi0NO46AnLj5np0k5U922GB0=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'HqzydB/ZjvAHG6hGKJXNX4Z1N5hwjhur4PgrntomRGPFl8ygMqpJi0NO46AnLj5np0k5U922GB0=', 'x-amz-request-id': 'JWM7XZSHVEEM884P', 'date': 'Mon, 25 Nov 2024 05:36:04 GMT', 'x-amz-server-side-encryption': 'AES256', 'etag': '"eed87956ea182f6fdc55059ac031762a"', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'ETag': '"eed87956ea182f6fdc55059ac031762a"', 'ServerSideEncryption': 'AES256'}

In [52]:
# 3.2. 10 ciudades con más casos de covid en Colombia ordenados de mayor a menor.
df_grouped_cities = df.groupBy("Nombre municipio").agg(count("ID de caso").alias("total_casos"))

df_sorted_cities = df_grouped_cities.orderBy(col("total_casos").desc())

df_top_10_cities = df_sorted_cities.limit(10)
df_top_10_cities.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-----------+
|Nombre municipio|total_casos|
+----------------+-----------+
|          BOGOTA|        401|
|            CALI|        101|
|        MEDELLIN|         63|
|       CARTAGENA|         39|
|    BARRANQUILLA|         31|
|           NEIVA|         27|
|         PEREIRA|         25|
|         PALMIRA|         22|
|      VALLEDUPAR|         16|
|         ARMENIA|         15|
+----------------+-----------+

In [53]:
df_pandas = df_top_10_cities.toPandas()
csv_buffer = StringIO()
df_pandas.to_csv(csv_buffer)
s3_client.put_object(Bucket='sboterodatasets', Key='top10_ciu_jupyter.csv', Body=csv_buffer.getvalue())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': 'YFRAHG625288EGTM', 'HostId': 'hXLvRUJatc6qdqoKxYPK1eKZf8ZZvnFV8BvjJZirjwwpvMuSbxoTVy69cW1bkDbrpNsafa8BsZ4=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'hXLvRUJatc6qdqoKxYPK1eKZf8ZZvnFV8BvjJZirjwwpvMuSbxoTVy69cW1bkDbrpNsafa8BsZ4=', 'x-amz-request-id': 'YFRAHG625288EGTM', 'date': 'Mon, 25 Nov 2024 05:36:11 GMT', 'x-amz-server-side-encryption': 'AES256', 'etag': '"b99aa2f9d13057067e127ee58aabfd63"', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'ETag': '"b99aa2f9d13057067e127ee58aabfd63"', 'ServerSideEncryption': 'AES256'}

In [54]:
# 3.3. Los 10 días con más casos de covid en Colombia ordenados de mayor a menor.
df_grouped_dates = df.groupBy("fecha reporte web").agg(count("ID de caso").alias("total_casos"))

df_sorted_dates = df_grouped_dates.orderBy(col("total_casos").desc())

df_top_10_dates = df_sorted_dates.limit(10)

df_top_10_dates.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-----------+
|fecha reporte web|total_casos|
+-----------------+-----------+
|31/3/2020 0:00:00|        107|
|24/3/2020 0:00:00|        105|
|30/3/2020 0:00:00|         96|
| 1/4/2020 0:00:00|         95|
|29/3/2020 0:00:00|         94|
|23/3/2020 0:00:00|         73|
|28/3/2020 0:00:00|         69|
|25/3/2020 0:00:00|         61|
|27/3/2020 0:00:00|         49|
|20/3/2020 0:00:00|         48|
+-----------------+-----------+

In [60]:
df_pandas = df_top_10_dates.toPandas()
csv_buffer = StringIO()
df_pandas.to_csv(csv_buffer)
s3_client.put_object(Bucket='sboterodatasets', Key='top10_days_jupyter.csv', Body=csv_buffer.getvalue())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': '0ZDWYXEGYN9S2D5J', 'HostId': 'CFJ/fTS3woIUGCo8YZAt2DMdnyPm3akYFhXjDLcRF24MussOOxSQP9ZUNnmpxjZpgH0tPIzsjDQ=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'CFJ/fTS3woIUGCo8YZAt2DMdnyPm3akYFhXjDLcRF24MussOOxSQP9ZUNnmpxjZpgH0tPIzsjDQ=', 'x-amz-request-id': '0ZDWYXEGYN9S2D5J', 'date': 'Mon, 25 Nov 2024 05:43:40 GMT', 'x-amz-server-side-encryption': 'AES256', 'etag': '"e6687f1979556d9d5cfb84e4074df1d1"', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'ETag': '"e6687f1979556d9d5cfb84e4074df1d1"', 'ServerSideEncryption': 'AES256'}

In [56]:
# 3.4. Distribución de casos por edades de covid en Colombia
df_age_distribution = df.groupBy("Edad").agg(count("ID de caso").alias("total_casos"))

df_age_distribution_sorted = df_age_distribution.orderBy(col("Edad").asc())

df_age_distribution_sorted.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------+
|Edad|total_casos|
+----+-----------+
|   1|          1|
|   2|          4|
|   3|          1|
|   4|          1|
|   5|          1|
|   6|          2|
|   8|          1|
|   9|          5|
|  12|          2|
|  13|          1|
|  15|          5|
|  16|          6|
|  17|          3|
|  18|         11|
|  19|         11|
|  20|          6|
|  21|         22|
|  22|         15|
|  23|         16|
|  24|         21|
+----+-----------+
only showing top 20 rows

In [57]:
df_pandas = df_age_distribution_sorted.toPandas()
csv_buffer = StringIO()
df_pandas.to_csv(csv_buffer)
s3_client.put_object(Bucket='sboterodatasets', Key='age_distribution_jupyter.csv', Body=csv_buffer.getvalue())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': '6G0A7G0TTW0EVVNV', 'HostId': '47jE9qstIjatywNrxIbiywISgn8yvwBvXm7v5na1qU0VEefDcM4j+gQrLPBSIjFRfe1C4KZWPrkRSY7EKZ3yoIXso9aEQuMtQKiPS8AB4F0=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': '47jE9qstIjatywNrxIbiywISgn8yvwBvXm7v5na1qU0VEefDcM4j+gQrLPBSIjFRfe1C4KZWPrkRSY7EKZ3yoIXso9aEQuMtQKiPS8AB4F0=', 'x-amz-request-id': '6G0A7G0TTW0EVVNV', 'date': 'Mon, 25 Nov 2024 05:36:40 GMT', 'x-amz-server-side-encryption': 'AES256', 'etag': '"01df95bbb82c0db6e30f4f0761561a83"', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'ETag': '"01df95bbb82c0db6e30f4f0761561a83"', 'ServerSideEncryption': 'AES256'}

In [58]:
# 3.5. Realice la pregunta de negocio que quiera sobre los datos y respondala con la correspondiente programación en spark.
# ¿Cómo ha sido la distribución de casos por género?

df_casos_por_genero = df.groupBy("Sexo").agg(count("ID de caso").alias("total_casos"))

# Mostrar los resultados
df_casos_por_genero.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------+
|Sexo|total_casos|
+----+-----------+
|   M|        520|
|   F|        480|
+----+-----------+

In [59]:
df_pandas = df_age_distribution_sorted.toPandas()
csv_buffer = StringIO()
df_pandas.to_csv(csv_buffer)
s3_client.put_object(Bucket='sboterodatasets', Key='genre_jupyter.csv', Body=csv_buffer.getvalue())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'ResponseMetadata': {'RequestId': '71NHAKDY5KFWEC0B', 'HostId': 'ENT++x16la4JqidyMsc7JU3koe21BEQBxbl6crN9qsItMeRCk6PCF6NYwtCQv++eEPT5/10bKzJY3G+0MIAGxMFTexv0VuBX95mIiY1IHuM=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'ENT++x16la4JqidyMsc7JU3koe21BEQBxbl6crN9qsItMeRCk6PCF6NYwtCQv++eEPT5/10bKzJY3G+0MIAGxMFTexv0VuBX95mIiY1IHuM=', 'x-amz-request-id': '71NHAKDY5KFWEC0B', 'date': 'Mon, 25 Nov 2024 05:36:44 GMT', 'x-amz-server-side-encryption': 'AES256', 'etag': '"01df95bbb82c0db6e30f4f0761561a83"', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'ETag': '"01df95bbb82c0db6e30f4f0761561a83"', 'ServerSideEncryption': 'AES256'}